# Practical Exam: Hotel Operations

LuxurStay Hotels is a major, international chain of hotels. They offer hotels for both business and leisure travellers in major cities across the world. The chain prides themselves on the level of customer service that they offer. 

However, the management has been receiving complaints about slow room service in some hotel branches. As these complaints are impacting the customer satisfaction rates, it has become a serious issue. Recent data shows that customer satisfaction has dropped from the 4.5 rating that they expect. 

You are working with the Head of Operations to identify possible causes and hotel branches with the worst problems. 

## Data

The following schema diagram shows the tables available. You have only been provided with data where customers provided a feedback rating.

![hotel_operations](hotel_operations.png)

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `branch` table, and the data team have provided the following data description. 

Write a query to return data matching this description. You must match all column names and description criteria.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|id | Nominal. The unique identifier of the hotel. </br>Missing values are not possible due to the database structure.|
| location | Nominal. The location of the particular hotel. One of four possible values, 'EMEA', 'NA', 'LATAM' and 'APAC'. </br>Missing values should be replaced with “Unknown”. |
| total_rooms | Discrete. The total number of rooms in the hotel. Must be a positive integer between 1 and 400. </br>Missing values should be replaced with the default number of rooms, 100. |
| staff_count | Discrete. The number of staff employeed in the hotel service department. </br>Missing values should be replaced with the total_rooms multiplied by 1.5. |
| opening_date | Discrete. The year in which the hotel opened. This can be any value between 2000 and 2023. </br>Missing values should be replaced with 2023. |
| target_guests | Nominal. The primary type of guest that is expected to use the hotel. Can be one of 'Leisure' or 'Business'. </br>Missing values should be replaced with 'Leisure'. |

In [6]:
-- Write your query for task 1 in this cell
SELECT 
    id,
    COALESCE(location, 'Unknown') AS location,
    CASE
        WHEN total_rooms BETWEEN 1 AND 400 THEN total_rooms
        ELSE 100
    END AS total_rooms,
    CASE
        WHEN staff_count IS NOT NULL THEN staff_count
        ELSE total_rooms * 1.5
    END AS staff_count,
    CASE
        WHEN opening_date = '-' THEN '2023'
        WHEN opening_date BETWEEN '2000' AND '2023' THEN opening_date
        ELSE '2023'
    END AS opening_date,
    CASE
        WHEN target_guests IS NULL THEN 'Leisure'
        WHEN LOWER(target_guests) LIKE 'b%' THEN 'Business'
        ELSE target_guests END AS target_guests
FROM 
    public.branch;

,id,location,total_rooms,staff_count,opening_date,target_guests
0,1,LATAM,168,178,2017,Business
1,2,APAC,154,82,2010,Leisure
2,3,APAC,212,467,2003,Leisure
3,4,APAC,230,387,2023,Business
4,5,APAC,292,293,2002,Business
...,...,...,...,...,...,...
95,96,APAC,237,257,2000,Business
96,97,APAC,107,169,2005,Business
97,98,EMEA,196,126,2002,Leisure
98,99,APAC,242,251,2021,Business


# Task 2

The Head of Operations wants to know whether there is a difference in time taken to respond to a customer request in each hotel. They already know that different services take different lengths of time. 

Calculate the average and maximum duration for each branch and service. Your output should include the columns `service_id`, `branch_id`, `avg_time_taken` and `max_time_taken`. Values should be rounded to two decimal places where appropriate. 

In [7]:
-- Write your query for task 2 in this cell
SELECT service_id, 
		branch_id, 
		ROUND(AVG(time_taken), 2) AS avg_time_taken, 
		MAX(time_taken) AS max_time_taken
FROM public.request
GROUP BY service_id, branch_id;

,service_id,branch_id,avg_time_taken,max_time_taken
0,2,46,13.09,16
1,4,99,9.13,13
2,1,8,2.56,10
3,2,13,13.53,17
4,1,46,2.08,4
...,...,...,...,...
380,4,73,9.43,13
381,4,88,9.36,12
382,1,89,2.77,7
383,4,31,9.00,9


# Task 3

The management team want to target improvements in `Meal` and `Laundry` service in Europe (`EMEA`) and Latin America (`LATAM`). 

Write a query to return the `description` of the service, the `id` and `location` of the branch, the id of the request as `request_id` and the `rating` for the services and locations of interest to the management team. 

Use the original branch table, not the output of task 1. 

In [8]:
-- Write your query for task 3 in this cell
SELECT s.description, b.id, b.location,r.id AS request_id, rating
FROM Request AS r
INNER JOIN service AS s
ON s.id = r.service_id
INNER JOIN branch AS b 
ON b.id = r.branch_id
WHERE s.description IN ('Meal', 'Laundry')
AND b.location IN ('EMEA', 'LATAM');

,description,id,location,request_id,rating
0,Laundry,63,EMEA,3,4
1,Laundry,69,LATAM,6,5
2,Meal,44,EMEA,18,4
3,Laundry,57,LATAM,19,3
4,Meal,1,LATAM,21,4
...,...,...,...,...,...
5042,Meal,30,EMEA,17662,4
5043,Meal,64,LATAM,17669,4
5044,Meal,51,LATAM,17674,5
5045,Meal,23,EMEA,17681,5


# Task 4

So that you can take a more detailed look at the lowest performing hotels, you want to get service and branch information where the average rating for the branch and service combination is lower than 4.5 - the target set by management.  

Your query should return the `service_id` and `branch_id`, and the average rating (`avg_rating`), rounded to 2 decimal places.

In [9]:
-- Write your query for task 4 in this cell
SELECT service_id, branch_id, ROUND(AVG(rating), 2) AS avg_rating
FROM Request
GROUP BY service_id, branch_id
HAVING AVG(rating) < 4.5;

,service_id,branch_id,avg_rating
0,2,46,3.78
1,4,99,3.83
2,1,8,3.64
3,1,46,3.81
4,3,15,4.00
...,...,...,...
210,3,8,3.38
211,1,64,3.59
212,4,93,3.72
213,4,88,3.60
